# Loop 4 Analysis: Ensemble Approach from Multiple Sources

## Goal
Implement the ensemble approach from jonathanchan kernel - find the best configuration for each N from ALL available CSV files across snapshots.

## Key Insight
The target score (68.896973) might be achievable by combining the best per-N configurations from different sources. Different optimization runs may have found better solutions for different N values.

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from numba import njit
import math
from tqdm import tqdm

# Tree polygon template
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate bounding box score for a group of trees"""
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    """Remove 's' prefix from values"""
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

tx, ty = make_polygon_template()
print("Template loaded")

Template loaded


In [2]:
# Find ALL CSV files in snapshots
all_csv_files = glob.glob('/home/nonroot/snapshots/santa-2025/**/*.csv', recursive=True)
print(f"Found {len(all_csv_files)} CSV files in snapshots")

# Also check current code directory
code_csvs = glob.glob('/home/code/**/*.csv', recursive=True)
print(f"Found {len(code_csvs)} CSV files in /home/code")

all_files = all_csv_files + code_csvs
print(f"Total: {len(all_files)} CSV files to scan")

Found 3160 CSV files in snapshots
Found 7 CSV files in /home/code
Total: 3167 CSV files to scan


In [3]:
# Initialize best tracker for each N
best = {n: {'score': 1e300, 'data': None, 'src': None} for n in range(1, 201)}

# Scan all CSV files
valid_files = 0
for fp in tqdm(all_files, desc='Scanning CSV files'):
    try:
        df = pd.read_csv(fp)
    except Exception:
        continue
    
    # Check if it's a valid submission file
    if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
        continue
    
    valid_files += 1
    df = df.copy()
    
    # Extract N from id
    try:
        df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    except:
        continue
    
    # Score each N group
    for n, g in df.groupby('N'):
        if n < 1 or n > 200:
            continue
        
        try:
            xs = strip(g['x'].to_numpy())
            ys = strip(g['y'].to_numpy())
            ds = strip(g['deg'].to_numpy())
            
            if len(xs) != n:
                continue  # Invalid group
            
            sc = score_group(xs, ys, ds, tx, ty)
            
            if sc < best[n]['score']:
                best[n]['score'] = float(sc)
                best[n]['data'] = g.drop(columns=['N']).copy()
                best[n]['src'] = fp.split('/')[-1]
        except:
            continue

print(f"\nScanned {valid_files} valid submission files")

Scanning CSV files:   0%|          | 0/3167 [00:00<?, ?it/s]

Scanning CSV files:   0%|          | 1/3167 [00:00<13:12,  4.00it/s]

Scanning CSV files:   0%|          | 3/3167 [00:00<06:28,  8.15it/s]

Scanning CSV files:   0%|          | 4/3167 [00:00<06:26,  8.18it/s]

Scanning CSV files:   0%|          | 6/3167 [00:00<05:07, 10.29it/s]

...

Scanning CSV files: 100%|█████████▉| 3162/3167 [04:00<00:00, 13.96it/s]

Scanning CSV files: 100%|█████████▉| 3164/3167 [04:00<00:00, 13.95it/s]

Scanning CSV files: 100%|█████████▉| 3166/3167 [04:00<00:00, 13.94it/s]

Scanning CSV files: 100%|██████████| 3167/3167 [04:01<00:00, 13.14it/s]


Scanned 3156 valid submission files


In [4]:
# Calculate total score from best per-N
total_score = 0
missing = []
for n in range(1, 201):
    if best[n]['data'] is None:
        missing.append(n)
    else:
        total_score += best[n]['score']

print(f"Total ensemble score: {total_score:.6f}")
print(f"Target: 68.896973")
print(f"Gap: {total_score - 68.896973:.6f}")
print(f"Missing N values: {missing}")

# Show score breakdown by range
ranges = [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_score = sum(best[n]['score'] for n in range(start, end+1) if best[n]['data'] is not None)
    print(f"N={start}-{end}: {range_score:.4f}")

Total ensemble score: 27.414787
Target: 68.896973
Gap: -41.482186
Missing N values: []
N=1-10: 2.4194
N=11-50: 2.3008
N=51-100: 8.2843
N=101-150: 7.4724
N=151-200: 6.9379


In [5]:
# Show best sources for each N (top improvements)
print("\nBest sources per N (showing first 20):")
for n in range(1, 21):
    if best[n]['data'] is not None:
        print(f"N={n:3d}: score={best[n]['score']:.6f}, src={best[n]['src']}")


Best sources per N (showing first 20):
N=  1: score=0.661250, src=submission.csv
N=  2: score=0.338427, src=test_invalid.csv
N=  3: score=0.262632, src=test_invalid.csv
N=  4: score=0.199376, src=test_invalid.csv
N=  5: score=0.206839, src=test_invalid.csv
N=  6: score=0.173625, src=submission.csv
N=  7: score=0.157468, src=submission_v21.csv
N=  8: score=0.136883, src=submission.csv
N=  9: score=0.128142, src=submission.csv
N= 10: score=0.154792, src=submission.csv
N= 11: score=0.135039, src=submission.csv
N= 12: score=0.115983, src=submission.csv
N= 13: score=0.108253, src=submission.csv
N= 14: score=0.111515, src=submission.csv
N= 15: score=0.106120, src=submission.csv
N= 16: score=0.062169, src=submission.csv
N= 17: score=0.102560, src=submission.csv
N= 18: score=0.077631, src=submission.csv
N= 19: score=0.068570, src=submission.csv
N= 20: score=0.061771, src=submission.csv


In [6]:
# Compare with current submission
current_df = pd.read_csv('/home/submission/submission.csv')
current_df['N'] = current_df['id'].astype(str).str.split('_').str[0].astype(int)

current_scores = {}
for n, g in current_df.groupby('N'):
    xs = strip(g['x'].to_numpy())
    ys = strip(g['y'].to_numpy())
    ds = strip(g['deg'].to_numpy())
    current_scores[n] = score_group(xs, ys, ds, tx, ty)

current_total = sum(current_scores.values())
print(f"Current submission score: {current_total:.6f}")
print(f"Ensemble score: {total_score:.6f}")
print(f"Improvement: {current_total - total_score:.6f}")

# Show where improvements come from
print("\nImprovements by N (top 20):")
improvements = []
for n in range(1, 201):
    if best[n]['data'] is not None and n in current_scores:
        imp = current_scores[n] - best[n]['score']
        if imp > 1e-9:
            improvements.append((n, imp, best[n]['src']))

improvements.sort(key=lambda x: -x[1])
for n, imp, src in improvements[:20]:
    print(f"N={n:3d}: improvement={imp:.9f}, src={src}")

Current submission score: 70.626027
Ensemble score: 27.414787
Improvement: 43.211239

Improvements by N (top 20):
N= 24: improvement=0.348368973, src=ensemble.csv
N= 41: improvement=0.337621976, src=ensemble.csv
N= 31: improvement=0.336178916, src=ensemble.csv
N= 33: improvement=0.335999224, src=ensemble.csv
N= 46: improvement=0.335350914, src=ensemble.csv
N= 44: improvement=0.334851838, src=ensemble.csv
N= 49: improvement=0.332665814, src=ensemble.csv
N= 38: improvement=0.330862601, src=ensemble.csv
N= 45: improvement=0.329824793, src=ensemble.csv
N= 42: improvement=0.328931584, src=ensemble.csv
N= 35: improvement=0.326789588, src=ensemble.csv
N= 50: improvement=0.326465576, src=ensemble.csv
N= 48: improvement=0.325896072, src=ensemble.csv
N= 47: improvement=0.325390689, src=ensemble.csv
N= 34: improvement=0.325215150, src=ensemble.csv
N= 40: improvement=0.324973833, src=ensemble.csv
N= 39: improvement=0.324205795, src=ensemble.csv
N= 43: improvement=0.322179452, src=ensemble.csv
N= 2

In [7]:
# Create ensemble submission
if total_score < current_total:
    print("Creating ensemble submission...")
    rows = []
    for n in range(1, 201):
        if best[n]['data'] is not None:
            rows.append(best[n]['data'])
    
    ensemble_df = pd.concat(rows, ignore_index=True)
    ensemble_df['sn'] = ensemble_df['id'].str.split('_').str[0].astype(int)
    ensemble_df['si'] = ensemble_df['id'].str.split('_').str[1].astype(int)
    ensemble_df = ensemble_df.sort_values(['sn', 'si']).drop(columns=['sn', 'si'])
    ensemble_df = ensemble_df[['id', 'x', 'y', 'deg']]
    
    # Save
    ensemble_df.to_csv('/home/code/experiments/ensemble_best.csv', index=False)
    print(f"Saved ensemble to /home/code/experiments/ensemble_best.csv")
    print(f"Ensemble score: {total_score:.6f}")
else:
    print("No improvement from ensemble")

Creating ensemble submission...
Saved ensemble to /home/code/experiments/ensemble_best.csv
Ensemble score: 27.414787


In [ ]:
# The score of 27.41 is suspiciously low - likely invalid configurations with overlaps
# Let me check the N=2 configuration from test_invalid.csv

print("Checking N=2 from test_invalid.csv:")
n2_data = best[2]['data']
print(n2_data)

# Calculate the actual bounding box
xs = strip(n2_data['x'].to_numpy())
ys = strip(n2_data['y'].to_numpy())
ds = strip(n2_data['deg'].to_numpy())
print(f"\nPositions: x={xs}, y={ys}, deg={ds}")

# Check for overlaps using Shapely
from shapely.geometry import Polygon
from shapely import affinity

def get_tree_polygon():
    return Polygon([
        (0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
        (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
        (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5),
    ])

BASE_TREE = get_tree_polygon()

def create_tree(x, y, deg):
    tree = affinity.rotate(BASE_TREE, deg, origin=(0, 0))
    tree = affinity.translate(tree, x, y)
    return tree

trees = [create_tree(xs[i], ys[i], ds[i]) for i in range(len(xs))]
print(f"\nTree 0 bounds: {trees[0].bounds}")
print(f"Tree 1 bounds: {trees[1].bounds}")

# Check overlap
if trees[0].intersects(trees[1]) and not trees[0].touches(trees[1]):
    intersection = trees[0].intersection(trees[1])
    print(f"\nOVERLAP DETECTED! Intersection area: {intersection.area}")
else:
    print("\nNo overlap")